In [ ]:
import json
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
#Thrown when element could not be found.

from selenium.common.exceptions import TimeoutException
#Thrown when a command does not complete in enough time.

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
# Set of supported locator strategies.

from selenium.webdriver.chrome.options import Options
from tqdm import tqdm_notebook

In [ ]:
base_url = "https://www.trustpilot.com"

In [ ]:
def get_soup(url):
#     BeautifulSoup(requests.get(base_url).content, 'html.parser')
    return BeautifulSoup(requests.get(url).content, 'lxml')

In [ ]:
# from webdriver_manager.chrome import ChromeDriverManager
# driver = webdriver.Chrome(ChromeDriverManager().install())
# driver.get(base_url)
# driver.page_source

In [ ]:
data = {}

soup = get_soup(base_url+ '/categories')

In [ ]:
# a = {}
# a['ayush'] = {}
# a['ayush']['pratap']='bharat'
# a

In [ ]:
for category in soup.findAll('div', {'class': 'subCategory___BRUDy'}):
    name = category.find('h3', {'class':'subCategoryHeader___36ykD'}).text
    name = name.strip()#Return a copy of the string with leading and trailing whitespace remove.
    data[name] = {}
    sub_categories = category.find('div', {'class': 'subCategoryList___r67Qj'})
#     print(category.find('div', {'class':'subCategoryItem___3ksKz'}))
#     print(sub_categories)
    for sub_category in sub_categories.findAll('div', {'class': 'subCategoryItem___3ksKz'}):
        sub_category_name = sub_category.find('a', {'class': 'navigation___2Efid'}).text 
        sub_category_uri = sub_category.find('a', {'class': 'navigation___2Efid'})['href'] 
        data[name][sub_category_name] = sub_category_uri

In [ ]:
for i in data:
    print(i)

In [ ]:
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('start-maximized')
options.add_argument('disable-infobars')
options.add_argument("--disable-extensions")

prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)
# This prevents Selenium from opening up a Chrome window thus accelerating the scraping.

from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

# driver = webdriver.Chrome('./driver/chromedriver', options=options)
# driver = webdriver.Chrome('C:/Users/BHARAT PRATAP/.wdm/chromedriver/81.0.4044.138/win32/chromedriver', options=options)

timeout = 3

In [ ]:
import re
s = 'https://www.trustpilot.com/review/pawprintgenetics.com'

'review' in re.findall(r'\w+', s)
# re.findall(r'\w+:/{2}\w+.\w+.\w+/{1}\w+', s)

In [ ]:
data

In [ ]:
# data['Animals & Pets']['Animal Health']
# driver.get('https://www.trustpilot.com/categories/ball_games')#base_url+data['Animals & Pets']['Animal Health']+'?numberofreviews=0&timeperiod=0&status=all')
# BeautifulSoup(driver.page_source, 'lxml')
# l = driver.find_elements_by_xpath('//a[@class="wrapper___2rOTx"]')
# [a.get_attribute('href') for a in l if 'review' in re.findall(r'\w+', a.get_attribute('href'))]
# # l

In [ ]:
def extract_company_urls_form_page():
    a_list = driver.find_elements_by_xpath('//a[@class="wrapper___2rOTx"]')
    URLs = [a.get_attribute('href') for a in a_list if 'review' in re.findall(r'\w+', a.get_attribute('href'))]
    dedup_urls = list(set(URLs))
    return dedup_urls

In [ ]:
def go_next_page():
    try:
        button = driver.find_element_by_xpath('//a[@class="paginationLinkNormalize___scOgG paginationLinkNext___1LQ14"]')
        return True, button
    except NoSuchElementException:
        return False, None

In [ ]:
data.get('Animals & Pets')

In [ ]:
company_urls = {}

In [ ]:
base_url + data['Animals & Pets']['Animal Health']

In [ ]:
# for category in tqdm_notebook(data):
for sub_category in tqdm_notebook(data['Animals & Pets'], leave=False):
    company_urls[sub_category] = []
    url = base_url + data['Animals & Pets'][sub_category]
    driver.get(url)
#         try:
#             element = EC.presence_of_element_located((By.CLASS_NAME, 'wrapper___2rOTx'))
#             WebDriverWait(driver, timeout).until(element)
#         except:
#             pass
#         next_page = True
    c = 1
    while c<=5:#next_page:
        extracted_company_urls = extract_company_urls_form_page()
        company_urls[sub_category] += extracted_company_urls
        next_page, button = go_next_page()
        if next_page:
            c += 1
            next_url = base_url + data['Animals & Pets'][sub_category]+f'?page={c}'
            driver.get(next_url)
#         print(c)

In [ ]:
company_urls

In [ ]:
# df_administration_services = pd.DataFrame(company_urls,)

# df_administration_services.to_csv('administration_services.CSV', index=False)

In [ ]:
# company_urls = {}
# for category in tqdm_notebook(data):
#     for sub_category in tqdm_notebook(data[category], leave=False):
#         company_urls[sub_category] = []

#         url = base_url + data[category][sub_category] + "?numberofreviews=0&timeperiod=0&status=all"
#         driver.get(url)
#         try: 
#             element_present = EC.presence_of_element_located(
#                 (By.CLASS_NAME, 'wrapper___2rOTx'))
            #https://selenium-python.readthedocs.io/waits.html
#             WebDriverWait(driver, timeout).until(element_present)
#         except:
#             pass
    
#         next_page = True
#         c = 1
#         while next_page:
#             extracted_company_urls = extract_company_urls_form_page()
#             company_urls[sub_category] += extracted_company_urls
#             next_page, button = go_next_page()
            
#             if next_page:
#                 c += 1
#                 next_url = base_url + data[category][sub_category] + "?numberofreviews=0&timeperiod=0&status=all" + f'?page={c}'
#                 driver.get(next_url)
#                 try: 
#                     element_present = EC.presence_of_element_located(
#                         (By.CLASS_NAME, 'wrapper___2rOTx'))
                    
#                     WebDriverWait(driver, timeout).until(element_present)
#                 except:
#                     pass


In [ ]:
consolidated_data = []

for category in data:
    for sub_category in data[category]:
        for url in company_urls[sub_category]:
            consolidated_data.append((category, sub_category, url))

df_consolidated_data = pd.DataFrame(consolidated_data, columns=['category', 'sub_category', 'company_url'])

df_consolidated_data.to_csv('consolidate_company_urls.CSV', index=False)

In [ ]:
company_urls